In [4]:
import json
import html
import re
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer


## Traitement NLP

Afin de réaliser l’embedding des avis, nous avons choisi d’utiliser SBERT. Ce modèle repose sur BERT, un encodeur Transformer, capable de capturer plusieurs couches d’information et donc les différentes nuances présentes dans un même texte. C’est particulièrement intéressant pour les avis Yelp, qui sont souvent riches et subjectifs.

Notre objectif n’est pas d’identifier le type de nourriture, mais plutôt l’ambiance et les caractéristiques des restaurants. Or, avec des méthodes plus simples, ce sont généralement les mots liés à la cuisine qui ressortent le plus, au détriment de ces aspects plus subtils.

Nous utilisons SBERT car, grâce au mécanisme de pooling, il est beaucoup plus rapide que BERT classique et directement adapté à la génération d’embeddings. D’après l’état de l’art, c’est aujourd’hui l’une des approches les plus efficaces pour représenter des textes courts comme des avis.

Source : https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html

In [5]:
data = []
with open("data/philly_restaurant_reviews.json", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
textes=pd.DataFrame(data)

In [6]:
# nombre d'avis par business
nb_avis = textes.groupby("business_id").size()
nb_avis.describe()

count    5852.000000
mean      117.445147
std       247.806219
min         5.000000
25%        14.000000
50%        40.000000
75%       118.000000
max      5778.000000
dtype: float64

On choisit de garder la mediane comme nombre d'avis par restaurant

On garde un nettoyage leger, sbert fonctionnant le mieux sur des phrases à langage humain

In [7]:
#nettoyer les données avant de les embedder
def clean_data(text):
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [8]:

# s'assurer que la colonne date est bien au format datetime
textes["date"] = pd.to_datetime(textes["date"])
textes["text_clean"] = textes["text"].apply(clean_data)

textes_40 = (
    textes
    .sort_values("date", ascending=False)   # du plus récent au plus ancien
    .groupby("business_id")
    .head(40)                                # garder les 40 plus récents
)


Nous utilisons le transformer model "all-MiniLM-L6-v2" utilisé par Bertopic. Les embeddings sont d’abord calculés au niveau des avis individuels, puis agrégés au niveau du restaurant .Les embeddings sont normalisés afin de travailler dans un espace vectoriel comparable. Nous calculons ensuite la moyenne des embeddings des avis, ce qui permet d’obtenir une représentation globale du restaurant. Enfin, cette moyenne est à nouveau normalisée pour garantir une norme unitaire, ce qui facilite les comparaisons entre restaurants.

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embedder chaque avis individuellement, puis moyenner par restaurant
print("Embedding des avis")
business_embeddings = []

for business_id in textes_40['business_id'].unique():
    business_reviews = textes_40[textes_40['business_id'] == business_id]['text_clean'].tolist()
    review_embeddings = model.encode(
        business_reviews,
        batch_size=32,
        show_progress_bar=True,
        normalize_embeddings=True
    )
    mean_embedding = review_embeddings.mean(axis=0)
    mean_embedding = mean_embedding / np.linalg.norm(mean_embedding)

    business_embeddings.append({
        'business_id': business_id,
        'embedding': mean_embedding,
        'n_reviews': len(business_reviews)
    })

textes_restaurant = pd.DataFrame(business_embeddings)


print(f"Dimension des embeddings: {textes_restaurant['embedding'].iloc[0].shape}")

textes_restaurant.head()

In [ ]:
textes_restaurant.to_csv('resultats_clustering_complet.csv', index=False)